In [1]:
#%qtconsole
import PyPDF2
import re
#import textract

In [2]:
#write a for-loop to open many files -- leave a comment if you'd #like to learn how
filename = 'data/LREC/LREC2012_Proceedings/pdf/106_Paper.pdf'
#open allows you to read the file
pdfFileObj = open(filename,'rb')
#The pdfReader variable is a readable object that will be parsed
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#discerning the number of pages will allow us to parse through all #the pages
num_pages = pdfReader.numPages
text = ""
#The for loop will read each page
for page_index in range(num_pages):
    pageObj = pdfReader.getPage(page_index)
    text += pageObj.extractText()

In [4]:
startindex1 = text.find('\n   Keywords:\n ') + len('\n   Keywords:\n ')
stopindex1 = text.find(' \n 1. Introduction \n')
text[startindex1:stopindex1].split(',')

['multimodal corpora', ' conversational video data', ' interaction engagement']

In [5]:
#text = text.replace('\n', '')
startindex2 = text.find(' \n 1. Introduction \n')
stopindex2 = re.search('\s*\n*\d*\.*\s(References|\w* References)\s+\n*', text, re.IGNORECASE).start()
body = text[startindex2:stopindex2]

In [ ]:
import bs4
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import sys

path = ".\\data\ACL\\"
if 'Apple' in sys.version:
    path = "data/ACL/"
url = 'http://aclweb.org/anthology/'
html = urlopen(url).read()
namecount = 1

soup = BeautifulSoup(html, 'html.parser')







L = ['J,', 'Q,', 'P,', 'E,', 'N,', 'D,', 'K,', 'S,', 'W,', 'A,', 'C,', 'H,', 'L,', 'Y,', 'O,', 'T']



pattern_old = r'http:\/\/aclweb\.org\/anthology\/[JQPENDKSWACHLYOT]\/[JQPENDKSWACHLYOT]\d\d\/'
pattern = r'[JQPENDKSWACHLYTUX]\/[JQPENDKSWACHLYTUX]\d\d\/'
tags = soup.findAll('a', href = re.compile(pattern))
links = []
for tag in tags:
    #print(url + tag.attrs['href'])
    links.append(url + tag.attrs['href'])


def download_file(download_url):
    global namecount
    response = urlopen(download_url)
    file = open('tmp.pdf', 'r+b')
    file.write(response.read())
    file.close()
    pdfFileObj = open('tmp.pdf', 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    num_pages = pdfReader.numPages
    text = ""
    #The for loop will read each page
    for page_index in range(num_pages):
        pageObj = pdfReader.getPage(page_index)
        text += pageObj.extractText()
    #with open(path + str(namecount) + '.txt', 'w') as f:
    with open(str(namecount) + '.txt', 'w') as f:
        namecount += 1
        f.write(text)
        

def get_pdf(links, baseURL=None):
    count = 0
        
    pdf_papers = []
    for link in links:
        if baseURL is None:
            baseURL = link
        html = urlopen(link).read()
        soup = BeautifulSoup(html, 'html.parser')
        pattern = r'\.pdf$'
        tags = soup.findAll('a', href = re.compile(pattern))
        
        for tag in tags:
            count += 1
            #print(baseURL.replace('.html', '/') + tag.attrs['href'])
            pdf_papers.append(baseURL + tag.attrs['href'])
            try:
                download_file(baseURL.replace('.html', '/') + tag.attrs['href'])
            except:
                pass
            
    return pdf_papers, count
  

pdfs, count1 = get_pdf(links)




endings = ['ANN', 'BIOMED', 'DAT', 'DIAL', 'FSM', 'GEN', 'HAN', 'HUM', 'LEX', 'MEDIA', 'MOL', 'MT', 'NLL', 'PARSE', 'MORPHON', 'SEM', 'SLAV', 'SEMITIC', 'SLPAT', 'UR', 'WAC']
endings = ['sig' + item.lower() for item in endings]

links = []
for ending in endings:
    link = url + ending + '.html'
    links.append(link)
pdfs, count2 = get_pdf(links, url)


print('number of pdf files: ', count1, count2, count1 + count2)


In [7]:
import os
os.getcwd()

'/Users/Friedi/OneDrive - bwedu/Unidokumente/Extraktion von Fachwortschatz aus Fachtexten/scrapeLREC'